## Import Packages

In [10]:
import os
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import numpy as np

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver

## Prepare

In [12]:
STORAGE_DIR = "../input"

In [14]:
OUTPUT_DIR = "../output"

In [16]:
DOC_DIR = "../docu"

### news paper url dictionaries

In [19]:
# List of news pages to be scraped
newspaper_urls = dict(
    sz="https://www.sueddeutsche.de/thema/Nahost",
    zeit="https://www.zeit.de/thema/israel",
    spiegel="https://www.spiegel.de/thema/nahost/p2/",
)

### target objects

In [22]:
content_dict = {}
text_dict = {}
log_list = []
failing_list = []

### date format

In [25]:
# Current date as string
now = datetime.now()
now_str = now.strftime("%Y-%m-%d")
print(now_str)

2025-11-21


### Data Extraction

#### HTML Roh-Datei exportieren

In [27]:
def scrape_website(name, url):
    # (1) Run request
    response = requests.get(url, allow_redirects=True)
    content = response.content
    text = response.text

    # (2) File name to store the raw HTML
    file_name = os.path.join(
        STORAGE_DIR,
        f"{now_str}-{name}.html",
    )

    # (3) Write raw HTML
    with open(file_name, "wb") as f:
        f.write(response.content)

    # (4) Fill content_dict and text_dict
    content_dict[name] = response.content
    text_dict[name] = response.text

    # (5) Fill log_list
    log_info = dict(
        name=name,
        date=now_str,
        file_name=file_name,
        status=response.status_code,
        url=response.url,
        encoding=response.encoding,
    )
    log_list.append(log_info)

**Ausführen**

In [29]:
for name, url in newspaper_urls.items():
    try:
        scrape_website(name, url)
    except:
        failing_list.append((name, url))

### sueddeutsche

In [26]:
sz="https://www.sueddeutsche.de/thema/Nahost"

In [117]:
sz_html = os.path.join("../input/2025-11-20-sz.html")

In [119]:
with open(sz_html, "r", encoding= "utf-8") as f:
    sz_text = f.read()

#### HTML-Seite iterieren

In [122]:
sz_soup = BeautifulSoup(sz_text, "html.parser")

In [124]:
sz_article_tag = sz_soup.select('article.sz-teaserlist-element')

In [126]:
article_id = []
title = []
content_preview = []
datum = []

for article in sz_article_tag:
    
    id_tag = article.attrs.get('id', 'KEINE ID GEFUNDEN')
    article_id.append(id_tag)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
    title_tag = article.select('[data-manual="teaser-title"]')

    if title_tag:

        for t in title_tag:
        
            title.append(t.get_text(strip=True))
                    
    else: 
        title.append('NA')

    # Vorschau des Artikel Inhaltes
#------------------------------------------------------------------------------------------------------    
    preview_tag = article.select('[data-manual="teaser-text"]')

    if preview_tag:
        for p in preview_tag:
        
            content_preview.append(p.get_text(strip=True))

    else:
        content_preview.append('NA')

    # Datum der Veröffentlichung
#------------------------------------------------------------------------------------------------------    
    datum_tag = article.select('span.css-0')

    
    if datum_tag:

        for d in datum_tag:
            
            datum.append(d.get_text(strip=True))

    else:
        datum.append('NA')

print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(title)
print(content_preview)
print(datum)

Gefundene Artikel: 151
['Menschenrechtler werfen Israel Kriegsverbrechen im Westjordanland vor', 'Ach, das bisschen Mord', 'Oberlandesgericht verhandelt Kriegsverbrechen in Syrien', 'Der Kronprinz hebt ab', 'Wenn der Außenminister innenpolitisch unter Beobachtung steht', 'Gentleman', 'Trump zum Khashoggi-Mord: „Dinge passieren“', 'Störfeuer gegen Trumps Gaza-Plan', '„Es gibt hier eine reale Macht der Islamisten“', 'Trumps Frieden oder kein Frieden', 'Rätselhafte Charterflüge nach Johannesburg', 'Israel erhält wieder deutsche Rüstungsexporte', 'Meine schönsten Gaza-Selfies', 'Araber dürfen früheren Dax-Konzern kaufen', 'Zwischenfälle in Gaza', 'EU-Kommission billigt Übernahme von Covestro', 'Wie Netanjahu die Demokratie angreift', 'Pro-palästinensische Aktivisten auf Brandenburger Tor', 'So bringt Deutschland die irreguläre Migration wieder unter Kontrolle', 'Ministerpräsident gewinnt Parlamentswahl', 'Klagen gegen Waffenlieferungen an Israel abgewiesen', 'IAEA: Iran verletzt Regeln', '

In [128]:
article_dict = {
    'id': article_id,
    'title': title,
    'preview': content_preview,
    'date': datum
}

In [130]:
sz_df = pd.DataFrame(article_dict)
sz_df

,id,title,preview,date
0,teaser-httpswwwsueddeutschedepolitiknahost-liv...,Menschenrechtler werfen Israel Kriegsverbreche...,Das Militär habe drei Flüchtlingslager gestürm...,NA
1,teaser-httpswwwsueddeutschedemeinungtrump-bin-...,"Ach, das bisschen Mord",Der US-Präsident verhöhnt ein Opfer der saudis...,19.11.2025
2,teaser-httpswwwsueddeutschedepolitikkoblenz-pr...,Oberlandesgericht verhandelt Kriegsverbrechen ...,"Fünf Männer sind angeklagt, weil sie während d...",19.11.2025
3,teaser-httpswwwsueddeutschedepolitikusa-bin-sa...,Der Kronprinz hebt ab,Donald Trump sichert dem saudischen Thronfolge...,19.11.2025
4,teaser-httpswwwsueddeutschedewirtschaftwadephu...,Wenn der Außenminister innenpolitisch unter Be...,Für Bundesaußenminister Johann Wadephul läuft ...,19.11.2025
...,...,...,...,...
146,teaser-httpswwwsueddeutschedepolitiktrump-frie...,"„Bibi, das ist deine Chance auf den Sieg“","US-Präsident Donald Trump glaubt, sein Frieden...",5.10.2025
147,teaser-httpswwwsueddeutschedeprojekteartikelpo...,Besuch in einer Stadt aus Trümmern,Die israelische Armee bringt Journalisten nach...,5.10.2025
148,teaser-httpswwwsueddeutschedepolitikgaza-flott...,Thunberg klagt über Wassermangel und Bettwanzen,Die Aktivisten der Gaza-Hilfsflottille sollen ...,5.10.2025
149,teaser-httpswwwsueddeutschedemeinungkommentar-...,Findet Netanjahu eine Exit-Strategie aus dem K...,NA,5.10.2025


#### HTML-Seite erweitern

In [ ]:
sz = "https://www.sueddeutsche.de/thema/Nahost"
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y")
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

try:
    wait = WebDriverWait(driver, 10)
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "sdd-cmp-modal-button-agree")) 
    )
    cookie_button.click()
    time.sleep(1) 
except TimeoutException:
    pass

date_selector = (By.CSS_SELECTOR, 'span.css-0')

while True:
    try:
        count_before = len(driver.find_elements(*date_selector))
    
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-1kk8krp"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", load_element)
        time.sleep(0.5) 

        driver.execute_script("arguments[0].click();", load_element)

        try:
            wait.until(
                lambda d: len(d.find_elements(*date_selector)) > count_before
            )
        except TimeoutException:
            break 
            
        all_dates = driver.find_elements(*date_selector)
        if not all_dates:
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                break
        except ValueError: 
            pass 

    except TimeoutException:
        break

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")

### Zeit

In [31]:
zeit = "https://www.zeit.de/thema/israel"

In [33]:
zeit_html = os.path.join("../input/2025-11-21-zeit.html")

In [35]:
with open(zeit_html, "r", encoding= "utf-8") as f:
    zeit_text = f.read()

In [37]:
zeit_driver = webdriver.Chrome()

#### HTML-Seite iterieren

In [39]:
zeit_soup = BeautifulSoup(zeit_text, "html.parser")

In [41]:
zeit_article_tag = zeit_soup.select('article.zon-teaser.zon-teaser--standard.kilkaya-all')

In [51]:
zeit_article_id = []
zeit_title = []
zeit_content_preview = []
zeit_datum = []

for zeit_article in zeit_article_tag:
    
    #id_tag = article.attrs.get('id', 'KEINE ID GEFUNDEN')
    #article_id.append(id_tag)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
    zeit_title_tag = zeit_article.select('span.zon-teaser__title')

    if zeit_title_tag:

        for t in zeit_title_tag:
        
            zeit_title.append(t.get_text(strip=True))
                    
    else: 
        zeit_title.append('NA')

    # Vorschau des Artikel Inhaltes
#------------------------------------------------------------------------------------------------------    
    zeit_preview_tag = zeit_article.select('p.zon-teaser__summary')

    if zeit_preview_tag:
        for p in zeit_preview_tag:
        
            zeit_content_preview.append(p.get_text(strip=True))

    else:
        zeit_content_preview.append('NA')

    # Datum der Veröffentlichung
#------------------------------------------------------------------------------------------------------    
    zeit_datum_tag = zeit_article.select('time.zon-teaser__datetime')

    
    if zeit_datum_tag:

        for d in zeit_datum_tag:
            
            zeit_datum.append(d.get_text(strip=True))

    else:
        zeit_datum.append('NA')

#print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(zeit_title)
print(zeit_content_preview)
print(zeit_datum)

['Der falsche Freund', 'EU-Außenminister beraten über Rolle in US-Friedensplan für Gaza', 'Reaktionen auf UN-Resolution zu Gazaplänen überwiegend positiv', 'UN-Sicherheitsrat sichert Gazafriedensplan ab', 'Deutschland hebt teilweisen Stopp von Rüstungsgütern für Israel auf', 'Was die Waffenruhe in Gaza nun torpedieren könnte', 'Dauerrausch am Abgrund', 'Zwischen Merz und Staatsräson', 'Siedlerangriffe im Westjordanland erreichen neuen Höchststand', 'Die abgebrochene Rede', 'Linkspartei wehrt sich gegen Vorwürfe zu Antisemitismus', 'Ein Crash mit Ansage', 'Israels Botschafter sieht linken Antisemitismus als größte Bedrohung', 'Gauck mahnt zu Wachsamkeit bei Antisemitismus aus arabischem Raum', 'Festnahmen nach Störung beim Konzert des Israel Philharmonic Orchestra', 'USA wollen Gaza-Friedensplan zügig durch UN-Sicherheitsrat bringen']
['Auch in Deutschland gibt es einen Hype um Zohran Mamdani, den kommenden Bürgermeister von New York City. Die Leute sollten genauer hinschauen.', 'Die EU

In [53]:
zeit_article_dict = {
    #'id': zeit_article_id,
    'title': zeit_title,
    'preview': zeit_content_preview,
    'date': zeit_datum
}

In [55]:
zeit_df = pd.DataFrame(zeit_article_dict)
zeit_df

,title,preview,date
0,Der falsche Freund,Auch in Deutschland gibt es einen Hype um Zohr...,Vor 7 Stunden
1,EU-Außenminister beraten über Rolle in US-Frie...,"Die EU erwägt laut einem Bericht, ihre Grenzsc...",Vor 1 Stunde
2,Reaktionen auf UN-Resolution zu Gazaplänen übe...,Nach der UN-Resolution zum Gazafriedensplan re...,18. November 2025
3,UN-Sicherheitsrat sichert Gazafriedensplan ab,Die Vereinten Nationen unterstützen den US-Pla...,18. November 2025
4,Deutschland hebt teilweisen Stopp von Rüstungs...,Die Bundesregierung will die Beschränkungen fü...,17. November 2025
5,Was die Waffenruhe in Gaza nun torpedieren könnte,"Geht es nach den USA, soll Gaza bald internati...",14. November 2025
6,Dauerrausch am Abgrund,Kann man die Folgen des 7. Oktober für Israel ...,14. November 2025
7,Zwischen Merz und Staatsräson,Der Thüringer CDU-Ministerpräsident Mario Voig...,13. November 2025
8,Siedlerangriffe im Westjordanland erreichen ne...,Palästinenser und Reporter werden zunehmend at...,12. November 2025
9,Die abgebrochene Rede,Der US-Philosoph Jason Stanley sollte vor der ...,11. November 2025


#### HTML-Seite erweitern

In [60]:
# 1. EINSTELLUNGEN
start_url = "https://www.zeit.de/thema/nahost"
# Wir suchen Artikel, die ÄLTER sind als dieser Tag (also bis zum 7.10. und davor)
target_date = datetime(2025, 10, 8) 

driver = webdriver.Chrome()
driver.get(start_url)

# 2. COOKIE BANNER (Versuch, ihn wegzuklicken)
try:
    wait = WebDriverWait(driver, 5)
    # Sucht nach "Zustimmen" oder "Akzeptieren"
    cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Zustimmen') or contains(text(), 'Akzeptieren')]")))
    cookie_btn.click()
    time.sleep(1)
    print("Cookie Banner geklickt.")
except:
    print("Kein Cookie Banner gefunden oder schon weg.")

all_articles = []
page_num = 1
stop_found = False

while True:
    print(f"--- Verarbeite Seite {page_num} ---")
    
    # HTML holen und Parsen
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles_on_page = soup.select('article.zon-teaser')
    
    if not articles_on_page:
        print("Keine Artikel auf dieser Seite gefunden.")
        break
        
    print(f"Gefundene Artikel auf dieser Seite: {len(articles_on_page)}")

    for art in articles_on_page:
        # Titel holen
        title_node = art.select_one('.zon-teaser__title')
        title = title_node.get_text(strip=True) if title_node else "NA"
        
        # Datum holen (aus dem datetime Attribut!)
        time_node = art.select_one('time.zon-teaser__datetime')
        
        date_obj = None
        date_str = "NA"
        
        if time_node and time_node.has_attr('datetime'):
            try:
                # Format ist meistens ISO: '2025-10-07T21:46:32+02:00'
                # Wir schneiden alles nach dem 'T' ab für den Vergleich
                date_str = time_node['datetime']
                iso_date_part = date_str.split('T')[0] 
                date_obj = datetime.strptime(iso_date_part, "%Y-%m-%d")
            except Exception as e:
                # Falls das Datum komisch formatiert ist, überspringen wir diesen einen Artikel
                continue
        
        # Zur Liste hinzufügen
        all_articles.append([title, date_str])
        
        # PRÜFUNG: Haben wir das Ziel erreicht?
        if date_obj and date_obj < target_date:
            print(f"!!! STOPP-DATUM ERREICHT: {date_obj.strftime('%d.%m.%Y')} bei Artikel: '{title}'")
            stop_found = True
            break 
    
    if stop_found:
        break

    # --- NÄCHSTE SEITE LADEN ---
    try:
        # Versuche den "Nächste Seite" Button zu finden
        # Strategie 1: CSS Klasse (Standard bei Zeit)
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.pager__button--next")
        
        # Scrollen, damit der Button nicht verdeckt ist
        driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
        time.sleep(1)
        
        # Klicken
        next_btn.click()
        
        # Warten, bis die URL sich ändert (z.B. auf ?p=2)
        page_num += 1
        WebDriverWait(driver, 10).until(EC.url_contains(f"p={page_num}"))
        
        # Kurze Pause für den Aufbau der Seite
        time.sleep(2)
        
    except (NoSuchElementException, TimeoutException):
        print("Kein 'Nächste Seite'-Button mehr gefunden oder Ende erreicht.")
        break

driver.quit()

# ERGEBNIS
print(f"\nGesamt gefundene Artikel: {len(all_articles)}")
if len(all_articles) > 0:
    print("Letzter gefundener Artikel:")
    print(all_articles[-1])

Kein Cookie Banner gefunden oder schon weg.
--- Verarbeite Seite 1 ---
Gefundene Artikel auf dieser Seite: 48
!!! STOPP-DATUM ERREICHT: 02.07.2025 bei Artikel: 'Iran und USA schließen zeitnahe Gespräche aus'

Gesamt gefundene Artikel: 37
Letzter gefundener Artikel:
['Iran und USA schließen zeitnahe Gespräche aus', '2025-07-02T11:36:11+02:00']


#### Spiegel

In [1008]:
spiegel="https://www.spiegel.de/thema/nahost/p2/"

In [1010]:
spiegel_html = os.path.join("../input/2025-11-07-spiegel.html")

In [1014]:
with open(spiegel_html, "r", encoding= "utf-8") as f:
    spiegel_text = f.read()

In [1016]:
spiegel_soup = BeautifulSoup(spiegel_text, "html.parser")

**Titel**

In [1024]:
# Large

driver.get(spiegel)

time.sleep(3) 


spiegel_html = driver.page_source
spiegel_soup = BeautifulSoup(spiegel_html, 'lxml')

spiegel_title_tag = spiegel_soup.select('span.align-middle')

print(spiegel_title_tag)

[<span class="inline-flex align-middle leading-none mr-4" data-contains-flags="plus-paid">
<span class="inline-block mr-4 text-primary-base dark:text-dm-primary-base" data-flag-name="plus-paid"><svg aria-hidden="true" class="fill-current" height="20" id="spon-spon-paid-flag-l" role="img" viewbox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg">
<g id="l-splus-flag-78f0aea1-16c7-48c9-84c4-c56919900756">
<rect height="20" id="bg-78f0aea1-16c7-48c9-84c4-c56919900756" rx="1.5" width="20"></rect>
<path clip-rule="evenodd" d="M4 10.684V6.295C4 5.575 4.36 5 5.295 5h3.669c.648 0 1.08.432 1.08 1.151v1.943H8.172v-1.44H5.799v2.303h4.244v4.82c0 .791-.431 1.223-1.295 1.223H5.223C4.36 15 4 14.496 4 13.849v-2.302h1.87v1.798h2.375v-2.662H4z" fill="#fff" fill-rule="evenodd" id="shape-78f0aea1-16c7-48c9-84c4-c56919900756"></path>
<path d="M13.825 7.8h1.35v2.025H17.2v1.35h-2.025V13.2h-1.35v-2.025H11.8v-1.35h2.025V7.8z" fill="#fff" id="shape_2-78f0aea1-16c7-48c9-84c4-c56919900756"></path>
</g>
</

In [1026]:
title_attr_df(spiegel_title_tag)

,titel
0,\n\n\n\n\n\n\n\n\n
1,Ziemlich beste Feinde
2,Israels Militär greift erneut Huthi-Miliz im J...
3,Rund 30 Verletzte bei israelischer Razzia in R...
4,Geiselangehörige warnen Netanyahu-Regierung vo...
5,Israelischer Ausschuss billigt umstrittenen Sp...
6,\n\n\n
7,"»Uns zu unterstellen, es ginge gar nicht um ec..."
8,Libanons Regierung will Waffenmonopol – Hisbol...
9,\n\n\n


**Content-Preview**

In [1039]:
spiegel_content_preview_tag = spiegel_soup.find_all('span', attrs={'data-target-teaser-el': True})

In [1034]:
title_attr_df(spiegel_content_preview_tag)

,titel
0,\nVor der Konferenz zur Zweistaatenlösung\n
1,\nSeit Monaten liefern sich Emmanuel Macron un...
2,"\nVon Leo Klimm, Paris\n"
3,\nKrieg in Nahost\n
4,\nImmer wieder attackieren die von Iran unters...
5,\nMilitäreinsatz im Westjordanland\n
6,\nIsraelische Soldaten sind zu einer Razzia in...
7,"\n»Hungrig, durstig, verängstigt, gefoltert«\n"
8,"\nSie bangen um das Überleben ihrer Söhne, Män..."
9,\nNeue Siedlungen\n


**Datum**

In [1041]:
spiegel_date_tag = spiegel_soup.find_all('span', attrs={'data-auxiliary': True})

In [1043]:
title_attr_df(spiegel_date_tag)

,titel
0,"22. September 2025, 19.28 Uhr"
1,"10. September 2025, 18.37 Uhr"
2,"27. August 2025, 12.42 Uhr"
3,"20. August 2025, 23.35 Uhr"
4,"20. August 2025, 14.16 Uhr"
5,"6. August 2025, 20.06 Uhr"
6,"6. August 2025, 08.21 Uhr"
7,"28. Juli 2025, 21.08 Uhr"
8,"27. Juli 2025, 13.50 Uhr"
9,"26. Juli 2025, 19.06 Uhr"


## Transform

## Test

In [1302]:
sz_df

,id,title,preview,date
0,teaser-httpswwwsueddeutschedepolitikbonn-iran-...,Iran will neues Konsulat in Bonn eröffnen,Das Mullah-Regime plant eine Vertretung am Rhe...,Heute 14:51 Uhr
1,teaser-httpswwwsueddeutschedeprojekteartikelpo...,„Ohne Staatsbürgerschaft hätte ich jetzt richt...,Bundeskanzler Merz sieht für Flüchtlinge aus S...,Heute 14:33 Uhr
2,teaser-httpswwwsueddeutschedepolitiksyrien-un-...,UN heben Sanktionen gegen al-Scharaa auf,Der UN-Sicherheitsrat hat die Sanktionen gegen...,Heute 13:36 Uhr
3,teaser-httpswwwsueddeutschedemeinungpalaestine...,NA,Ein 20-Punkte-Plan schafft noch keinen Frieden...,Heute 8:38 Uhr
4,teaser-httpswwwsueddeutschedepolitikunion-krit...,Herbst des Verdrusses,Eigentlich wollte Friedrich Merz mit einem Her...,6.11.2025
5,teaser-httpswwwsueddeutschedepolitiknahost-isr...,Israel greift im Südlibanon an,Israels Armee hat nach eigenen Angaben begonne...,6.11.2025
6,teaser-httpswwwsueddeutschedepolitikwadephul-u...,Wadephul gibt nicht klein bei,Der Außenminister verteidigt sich in der Union...,5.11.2025
7,teaser-httpswwwsueddeutschedemeinungkommentar-...,NA,Das Vertrauen in die Union sinkt immer mehr – ...,5.11.2025
8,teaser-httpswwwsueddeutschedepolitikisrael-fue...,Fünf Millionen Holocaust-Opfer von Yad Vashem ...,Die israelische Gedenkstätte Yad Vashem hat fü...,4.11.2025
9,teaser-httpswwwsueddeutschedemeinungsyrer-deut...,NA,"Fast alle sind sich einig, dass es richtig ist...",4.11.2025


In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time

from selenium.webdriver.support.ui import WebDriverWait

In [95]:
driver = webdriver.Chrome()
driver.get(sz)

In [62]:
load_element = driver.find_element(By.CLASS_NAME, 'css-1kk8krp')

In [99]:
# --- Dein Setup (Beispiel) ---
now_str = datetime.now().strftime("%Y-%m-%d")
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

#--------------------Laden der Seite--------------------
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y")
print(f"Suche gestartet. Stoppe, wenn Datum älter als {target_date.strftime('%d.%m.%Y')} ist.")

while True:
    try:
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, "css-1kk8krp"))
        )
        load_element.click()

        time.sleep(2) 

        all_dates = driver.find_elements(By.CSS_SELECTOR, 'span.css-0')
        if not all_dates:
            print("Keine Datumsangaben mehr gefunden.")
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                print(f"STOPP: {last_date_text} ist älter als das Ziel.")
                break
        except ValueError:
            pass 

    except TimeoutException:
        print("Button nicht mehr gefunden. Ende erreicht.")
        break


##--------------------Verlängerte Seite speichern--------------------

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")

Suche gestartet. Stoppe, wenn Datum älter als 08.10.2025 ist.
STOPP: 30.9.2025 ist älter als das Ziel.
Erfolgreich gespeichert in: ../input/2025-11-14-sz.html


In [55]:
driver = webdriver.Chrome()
driver.get(sz)

In [59]:
driver = webdriver.Chrome()
driver.get(sz)

button_element = driver.find_element(By.CLASS_NAME, 'css-1kk8krp')

datum_tag2 = article.select('span.css-0')
    
if datum_tag:

    for d in datum_tag:
            
        datum.append(d.get_text(strip=True))

else:
    datum2.append('NA')

for date in datum_tag2:
    date != "08.10.2025"
    button_element.click()

try:
    wait = WebDriverWait(driver, 10)
    load_element = wait.until(
        EC.element_to_be_clickable(load_element)
    )
except TimeoutException:
    print("Fehler: Button 'Mehr Artikel laden' wurde nach 10 Sek. nicht klickbar.")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=142.0.7444.176)
Stacktrace:
0   chromedriver                        0x000000010a3ce698 chromedriver + 6153880
1   chromedriver                        0x000000010a3c5b6a chromedriver + 6118250
2   chromedriver                        0x0000000109e5a878 chromedriver + 436344
3   chromedriver                        0x0000000109eae6bb chromedriver + 779963
4   chromedriver                        0x0000000109ea1a6a chromedriver + 727658
5   chromedriver                        0x0000000109ea13e8 chromedriver + 725992
6   chromedriver                        0x0000000109efbc8e chromedriver + 1096846
7   chromedriver                        0x0000000109e9fb6f chromedriver + 719727
8   chromedriver                        0x0000000109ea0871 chromedriver + 723057
9   chromedriver                        0x000000010a38b011 chromedriver + 5877777
10  chromedriver                        0x000000010a38f4c2 chromedr

In [66]:
load_element.click()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=142.0.7444.162); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001036549d8 chromedriver + 6482392
1   chromedriver                        0x000000010364be1a chromedriver + 6446618
2   chromedriver                        0x0000000103090afb chromedriver + 436987
3   chromedriver                        0x0000000103097add chromedriver + 465629
4   chromedriver                        0x000000010309a413 chromedriver + 476179
5   chromedriver                        0x000000010309a4e3 chromedriver + 476387
6   chromedriver                        0x00000001030e54fe chromedriver + 783614
7   chromedriver                        0x00000001030d7c2d chromedriver + 728109
8   chromedriver                 

In [115]:
sz = "https://www.sueddeutsche.de/thema/Nahost"
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y")
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

try:
    wait = WebDriverWait(driver, 10)
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "sdd-cmp-modal-button-agree")) 
    )
    cookie_button.click()
    time.sleep(1) 
except TimeoutException:
    pass

date_selector = (By.CSS_SELECTOR, 'span.css-0')

while True:
    try:
        count_before = len(driver.find_elements(*date_selector))
    
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-1kk8krp"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", load_element)
        time.sleep(0.5) 

        driver.execute_script("arguments[0].click();", load_element)

        try:
            wait.until(
                lambda d: len(d.find_elements(*date_selector)) > count_before
            )
        except TimeoutException:
            break 
            
        all_dates = driver.find_elements(*date_selector)
        if not all_dates:
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                break
        except ValueError: 
            pass 

    except TimeoutException:
        break

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")

Erfolgreich gespeichert in: ../input/2025-11-20-sz.html
